In [1]:
import numpy as np
from math import factorial as fact

In [2]:
d = 10

In [3]:
basis = []
for i in range(d-1):
    x = np.zeros(d-1).astype(int)
    x[i] = 1
    basis.append(x)

In [4]:
def partial(i,a):
    if i==2:
        return a - basis[0]
    return a + basis[i-3] - basis[i-2]

In [5]:
def partialterm(l,i,b):
    if i==2:
        return l-magn(b)+1
    return b[i-3]+1

In [6]:
def norm(a):
    return ((1+np.arange(len(a)))*a).sum()

def card(a):
    return a.sum()

def magn(a):
    return ((2+np.arange(len(a)))*a).sum()

In [74]:
def Lambda(l,a,b):
    if card(a)==0 or card(b)==0 or l<2:
        return fact(l-1)
    if norm(a)+norm(b)==l-1:
        if magn(a)<l and magn(b)<=l:
            answer = 0
            for i in range(2,l+1):
                if b[i-2] > 0:
                    answer += l*(i-1)*partialterm(l,i,b)*Lambda(l-1,a,partial(i,b))
                    print(l-magn(a),i,a,b,partial(i,b))
            return answer/(l-magn(a))
        elif magn(a)<=l and magn(b)<l:
            answer = 0
            for i in range(2,l+1):
                if a[i-2] > 0:
                    answer += l*(i-1)*partialterm(l,i,a)*Lambda(l-1,b,partial(i,a))
            return answer/(l-magn(b))
    return 0

In [75]:
lam = lambda l,a,b: Lambda(l,a,b)/fact(l-1)

In [77]:
lam(8,np.array([2,0,0,0,0]+4*[0]),np.array([0,0,0,0,1]+4*[0]))

1 2 [1 0 0 0 0 0 0 0 0] [1 0 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 0]
1 3 [2 0 0 0 0 0 0 0 0] [0 1 0 0 0 0 0 0 0] [1 0 0 0 0 0 0 0 0]
2 4 [2 0 0 0 0 0 0 0 0] [0 0 1 0 0 0 0 0 0] [0 1 0 0 0 0 0 0 0]
3 5 [2 0 0 0 0 0 0 0 0] [0 0 0 1 0 0 0 0 0] [0 0 1 0 0 0 0 0 0]
4 6 [2 0 0 0 0 0 0 0 0] [0 0 0 0 1 0 0 0 0] [0 0 0 1 0 0 0 0 0]


20.0

In [ ]:
lam(3,np.array([1,0,0,0,0]),np.array([1,0,0,0,0]))

In [23]:
b

NameError: name 'b' is not defined